# Better Data Splits for Machine Learning with `astartes`

acyclic molecules and boiling point trainset0
https://brunl01.users.greyc.fr/CHEMISTRY/

molplotly

In [2]:
import pandas as pd

with open("iupac_and_boiling.csv", "r") as f:
    data = pd.read_csv(f, sep="   ", engine="python")
data


,iupac_name,boiling_point
0,ethylmethylether,10.80
1,diethylether,34.60
2,isopropylmethylether,32.00
3,butylmethylether,70.30
4,ethylpropylether,63.60
...,...,...
122,"2,2,5,5-tetramethylhexane",137.46
123,"2,2,5-trimethylheptane",148.00
124,"2,5,5-trimethylheptane",152.80
125,"2,2,6-trimethylheptane",148.20


use py2opsin to convert the IUPAC names into smiles strings

In [3]:
from py2opsin import py2opsin

data["smiles"] = py2opsin(data["iupac_name"])


In [4]:
from astartes.molecules import train_val_test_split_molecules

import numpy as np

tts_args = dict(
    train_size=0.8,
    test_size=0.1,
    val_size=0.1,
    return_indices=True,
    fingerprint='padelpy:MW',
)

(
    molecules_train,
    molecule_val,
    molecules_test,
    random_idxs_train,
    random_idxs_val,
    random_idxs_test,
) = train_val_test_split_molecules(
    data["smiles"].to_numpy(),
    **tts_args,
)


/home/jackson/astartes/astartes/astartes/main.py:357: ImperfectSplittingWarning: Actual train/test split differs from requested size. Requested validation size of 0.10, got 0.11. Requested test size of 0.10, got 0.11. 
  warn(


array([['142.172150704'],
       ['128.15650064'],
       ['270.2922658360004'],
       ['270.2922658360004'],
       ['142.172150704'],
       ['128.15650064'],
       ['142.172150704'],
       ['142.172150704'],
       ['158.167065324'],
       ['144.15141526'],
       ['88.088815004'],
       ['102.104465068'],
       ['142.172150704'],
       ['130.135765196'],
       ['142.172150704'],
       ['102.104465068'],
       ['142.172150704'],
       ['102.104465068'],
       ['130.135765196'],
       ['102.104465068'],
       ['142.172150704'],
       ['142.172150704'],
       ['128.15650064'],
       ['60.057514876'],
       ['128.15650064'],
       ['128.15650064'],
       ['128.15650064'],
       ['102.104465068'],
       ['130.135765196'],
       ['214.22966558'],
       ['116.120115132'],
       ['116.120115132'],
       ['128.15650064'],
       ['142.172150704'],
       ['128.15650064'],
       ['186.198365452'],
       ['128.15650064'],
       ['116.120115132'],
       ['144.1514

In [23]:
molecules_train = np.array([float(i[0]) for i in molecules_train])
molecules_val = np.array([float(i[0]) for i in molecule_val])
molecules_test = np.array([float(i[0]) for i in molecules_test])


IndexError: invalid index to scalar variable.

In [15]:
molecules_MW = np.hstack((
    molecules_train,molecules_val,molecules_test
))
all_idxs = np.hstack((
    random_idxs_train,
    random_idxs_val,
    random_idxs_test,
))

In [78]:
import plotly.graph_objects as go
import plotly.express as px


fig = px.scatter(x=molecules_train, y=data["boiling_point"].to_numpy()[random_idxs_train], trendline="expanding", trendline_options=dict(function="max"),trendline_scope="overall", trendline_color_override="black")
fig.add_trace(go.Scatter(x=molecules_val, y=data["boiling_point"].to_numpy()[random_idxs_val],
                    mode='markers',
                    name='validation'))
fig.add_trace(go.Scatter(x=molecules_test, y=data["boiling_point"].to_numpy()[random_idxs_test],
                    mode='markers',
                    name='testing'))
fig.update_layout(
    dict(
        plot_bgcolor="rgba(0, 0, 0, 0)",
        paper_bgcolor="rgba(0, 0, 0, 0)",
        height=800,
        width=800,
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
    ),
)
ax_args = dict(
    showline=True, linewidth=2, linecolor='grey', gridwidth=1, showgrid=True, gridcolor="grey", zerolinecolor="grey", zerolinewidth=1
)
fig.update_xaxes(title="Molecular Weight", **ax_args)
fig.update_yaxes(title="Boiling Point (°C)", **ax_args)
# import warnings, molplotly
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore", category=FutureWarning)
#     app = molplotly.add_molecules(
#         fig=fig,
#         df=split_df,
#         show_coords=False,
#         color_col="split",
#         caption_cols=["Boiling Point (°C)"],
#         title_col="iupac_name",
#     )
fig.show()

In [5]:
import plotly.express as px
import molplotly
import warnings


def scatterplot(idxs_train, idxs_val, idxs_test, plot_port):
    split_membership = np.empty(len(data), dtype=object)
    split_membership[idxs_val] = "val"
    split_membership[idxs_train] = "train"
    split_membership[idxs_test] = "test"
    all_indxs = np.hstack(
        (
            idxs_train,
            idxs_val,
            idxs_test,
        )
    )
    split_df = pd.DataFrame.from_dict(
        {
            "MW": molecules_MW[all_indxs],
            "SMILES": data["smiles"].to_numpy()[all_indxs],
            "Boiling Point (°C)": data["boiling_point"].to_numpy()[all_indxs],
            "split": split_membership,
            "iupac_name": data["iupac_name"].to_numpy()[all_indxs],
        }
    )
    fig = px.scatter_3d(
        data_frame=split_df,
        x="PC1",
        y="PC2",
        z="Boiling Point (°C)",
        opacity=0.8,
        color="split",
        width=1000,
        height=1000,
    )
    fig.update_traces(
        marker=dict(size=4),
        selector=dict(mode="markers"),
        hovertemplate=None,  # moltplotly break without this
    )
    axis_args = dict(
        backgroundcolor="rgba(0, 0, 0,0)",
        gridcolor="grey",
        showbackground=True,
        zerolinecolor="grey",
        showticklabels=True,
    )
    fig.update_layout(
        dict(
            plot_bgcolor="rgba(0, 0, 0, 0)",
            paper_bgcolor="rgba(0, 0, 0, 0)",
        ),
        scene=dict(
            xaxis=axis_args,
            yaxis=axis_args,
            zaxis=axis_args,
            xaxis_title="PC1",
            yaxis_title="PC2",
            zaxis_title="Boiling Point (°C)",
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
        ),
    )
    consistent_colors = {"train": "grey", "val": "red", "test": "blue"}
    for i, d in enumerate(fig.data):
        fig.data[i].marker.color = consistent_colors[fig.data[i].name]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)
        app = molplotly.add_molecules(
            fig=fig,
            df=split_df,
            show_coords=False,
            color_col="split",
            caption_cols=["Boiling Point (°C)"],
            title_col="iupac_name",
        )
    app.run_server(mode="inline", port=plot_port, height=1000, width=1000)

In [6]:
scatter3d(random_idxs_train, random_idxs_val, random_idxs_test, 8700)


Dash is running on http://127.0.0.1:8700/



This embedding clusters the data into heavy alkanes with quaternary carbons, heavy alkanes with sub-quaternary carbons, light linear alkanes, and gases.

Dataset partitioning is done with the entire fingerprint, this project is just for our understanding.

In [7]:
from astartes import train_val_test_split
embedding_split_args = dict(
    train_size=0.8,
    test_size=0.1,
    val_size=0.1,
    return_indices=True,
)

(
    _,
    _,
    _,
    ks_idxs_train,
    ks_idxs_val,
    ks_idxs_test,
) = train_val_test_split(
    embedding,
    sampler="kennard_stone",
    hopts=dict(
        metric="euclidean",
    ),
    **embedding_split_args,
)
scatter3d(ks_idxs_train, ks_idxs_val, ks_idxs_test, 8701)

Dash is running on http://127.0.0.1:8701/



/home/jackson/astartes/astartes/astartes/main.py:357: ImperfectSplittingWarning:

Actual train/test split differs from requested size. Requested validation size of 0.10, got 0.11. Requested test size of 0.10, got 0.11. 



In [8]:
(
    _,
    _,
    _,
    _,
    _,
    _,
    spxy_idxs_train,
    spxy_idxs_val,
    spxy_idxs_test,
) = train_val_test_split(
    embedding,
    data["boiling_point"].to_numpy(),
    sampler="spxy",
    hopts=dict(
        metric="euclidean",
    ),
    **embedding_split_args,
)
scatter3d(spxy_idxs_train, spxy_idxs_val, spxy_idxs_test, 8702)

/home/jackson/astartes/astartes/astartes/main.py:357: ImperfectSplittingWarning:

Actual train/test split differs from requested size. Requested validation size of 0.10, got 0.11. Requested test size of 0.10, got 0.11. 



Dash is running on http://127.0.0.1:8702/



In [11]:
(
    _,
    _,
    _,
    clsuters_train,
    clusters_val,
    clusters_test,
    spex_idxs_train,
    spex_idxs_val,
    spex_idxs_test,
) = train_val_test_split(
    embedding,
    sampler="sphere_exclusion",
    hopts=dict(
        # n_clusters=15,
        metric="euclidean",
        distance_cutoff=0.1,
    ),
    random_state=122,
    **embedding_split_args,
)
scatter3d(spex_idxs_train, spex_idxs_val, spex_idxs_test, 8703)

Dash is running on http://127.0.0.1:8703/



/home/jackson/astartes/astartes/astartes/main.py:357: ImperfectSplittingWarning:

Actual train/test split differs from requested size. Requested train size of 0.80, got 0.81. Requested validation size of 0.10, got 0.09. Requested test size of 0.10, got 0.09. 

